# 01 - Practical Session - Supervised learning

## Import libraries

In [ ]:
# numpy for matrix operation, backend is implemented in c
import numpy as np

# matplotlib for plotting, interface is inspired by matlab
import matplotlib.pyplot as plt

# for 3D plots
from mpl_toolkits.mplot3d import Axes3D

# Data structures and analysis tools, we use it for plotting nice looking tables
import pandas as pd

# Machine learning framework
from sklearn import datasets

# matplotlib backend. An other choice could be inline. Notebook provieds interactive plots
%matplotlib inline

## Datasets

We need some datasets, we can work with. The first one we want to look at is the iris dataset[1,2].

[1] http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

[2] https://en.wikipedia.org/wiki/Iris_flower_data_set

In [ ]:
iris = datasets.load_iris()

Scikit learn provides already some common datasets we can load through the helper functions in the dataset module. The functions returns an object with several member variables holding the data.

The data member variable is a [n_samples, n_features] dimensional numpy.ndarray containing *n_samples* observations. For this dataset each feature vector in this matrix describes the following features [petal length, petal width, sepal leangth, sepal width]. 

In [ ]:
iris_x = iris.data

We can select the first 5 samples by *slicing* the array.

In [ ]:
iris_x[:5]

If we want to select only the first feature dimension, we have to index the second dimension.

In [ ]:
iris_x[:5,0]

The target member variable is a [n_samples] dimensional numpy.ndarray that contains the corresponding target values we are going to predict. The target classes are either 1, 2 or 3.

In [ ]:
iris_y = iris.target

One other popular choice is the digits dataset[1,2]. This dataset contains 1797 samples of 8x8 dimensional grayscale images of numbers from 0 to 9.

[1] http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html

[2] http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

In [ ]:
digits = datasets.load_digits()
digits_i = digits.images
digits_x = digits.data
digits_y = digits.target

One last dataset we want to look at is the diabetes dataset[1,2].

[1] http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html

[2] http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

In [ ]:
diabetes = np.loadtxt('datasets/diabetes.txt', skiprows=1)

In [ ]:
diabetes_x = diabetes[:,:-1]
diabetes_y = diabetes[:,-1]
diabetes_feature_names = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
diabetes_target_names = ['disease_progression']

### Visualization

Before you throw the dataset in your favorite learning algorithm, it is always a good idea to first inspect the data and get a feeling for it. 

In [ ]:
iris_y_labels = np.apply_along_axis(lambda i: iris.target_names[i], 0, iris_y)[:,None]
iris_data = np.concatenate([iris_x, iris_y_labels], axis=1)

In [ ]:
pd.DataFrame(iris_data[::10], columns=iris.feature_names + ['labels'])

Sometimes, visualizing the data can reveal already some structure that can be exploited in your learning algorithm or based on your inspection of the data you might decide if and how you should preprocess your data. 

**Exercise 1.1:** Plot the first feature of the iris dataset against the second feature as scatter plot. Color the data points based on their class: 

In [ ]:
# extract the first and second feature
iris_x_fst_feature = 
iris_x_snd_feature = 

# create a new figure object
fig = plt.figure()


# [1.1]: select datapoints 0 to 49 of fst feature
# [1.2]: select datapoints 0 to 49 of snd feature
_ = plt.scatter('''[1.1]''', '''[1.1]''', c='r', label=iris.target_names[0])

# [2.1]: select datapoints 50 to 99 of fst feature
# [2.2]: select datapoints 50 to 99 of snd feature
_ = plt.scatter('''[2.1]''', '''[2.1]''', c='b', label=iris.target_names[1])

# [3.1]: select datapoints 100 to end of fst feature
# [3.2]: select datapoints 100 to end of snd feature
_ = plt.scatter('''[3.1]''', '''[3.1]''', c='g', label=iris.target_names[2])

_ = plt.xlabel('1st feature ({})'.format(iris.feature_names[0]))
_ = plt.ylabel('2st feature ({})'.format(iris.feature_names[1]))
_ = plt.title('Scatter plot of the first two components of the iris dataset')
_ = plt.legend()

In [ ]:
# extract the first and second feature
iris_x_fst_feature = iris_x[:,0]
iris_x_snd_feature = iris_x[:,1]

# create a new figure object
fig = plt.figure()

# [1.1]: select datapoints 0 to 49 of fst feature
# [1.2]: select datapoints 0 to 49 of snd feature
_ = plt.scatter(iris_x_fst_feature[:50], iris_x_snd_feature[:50], c='r', label=iris.target_names[0])

# [2.1]: select datapoints 50 to 99 of fst feature
# [2.2]: select datapoints 50 to 99 of snd feature
_ = plt.scatter(iris_x_fst_feature[50:100], iris_x_snd_feature[50:100], c='b', label=iris.target_names[1])

# [3.1]: select datapoints 100 to end of fst feature
# [3.2]: select datapoints 100 to end of snd feature
_ = plt.scatter(iris_x_fst_feature[100:], iris_x_snd_feature[100:], c='g', label=iris.target_names[2])

_ = plt.xlabel('1st feature ({})'.format(iris.feature_names[0]))
_ = plt.ylabel('2st feature ({})'.format(iris.feature_names[1]))
_ = plt.title('Scatter plot of the first two components of the iris dataset')
_ = plt.legend()

Visualization of instances from different classes of the digits dataset (inter-class variability)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(10,5))
_ = [axs.flat[i].imshow(digits_i[i], cmap='gray') for i in range(0,10)]

Visualization of instances from the same class of the digits dataset (inter-class variability)

In [ ]:
zeros = digits_i[np.where(digits_y==0)]

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(10,5))
_ = [axs.flat[i].imshow(zeros[i], cmap='gray') for i in range(0,10)]

For high dimensional data unsupervised dimensionality reductions methods like PCA or t-SNE can be used to reveal structure in the dataset. We will talk about some of these methods in the next session.

### Preprocessing

#### Normalization

In case that your dataset consists of features of very different kind it might easily happen that different channels cover quite different ranges.

In [ ]:
fig = plt.figure()
_ = plt.hist(diabetes_x[:,0], label='age')
_ = plt.hist(diabetes_x[:,2], alpha=0.66, label='bmi')
_ = plt.hist(diabetes_x[:,4], alpha=0.33, label='s1')
_ = plt.legend()

In order to weight the contribution of each feature equally, it is recommended to normalize each input channel by subtracting the mean and divide by the standard deviation over all training samples.

**Exercise 1.1:** Fill in the blank fields. Normalize each feature channel to zero center and unit variance as well as normalize each feature channel such that the square sum equals one: 

In [ ]:
diabetes_x_mean = 
diabetes_x_std = 
diabetes_x_norm = 

In [ ]:
diabetes_x_mean = np.mean(diabetes_x, axis=0)
diabetes_x_std = np.std(diabetes_x, axis=0)
diabetes_x_norm = (diabetes_x - diabetes_x_mean) / (diabetes_x_std)
diabetes_x_norm = diabetes_x_norm / np.sqrt(np.sum(diabetes_x_norm**2, axis=0))

In [ ]:
fig = plt.figure()
_ = plt.hist(diabetes_x_norm[:,0], label='age')
_ = plt.hist(diabetes_x_norm[:,2], alpha=0.66, label='bmi')
_ = plt.hist(diabetes_x_norm[:,4], alpha=0.33, label='s1')
_ = plt.legend()

Scikit learn provides convenient functions to do the job

In [ ]:
from sklearn import preprocessing

In [ ]:
diabetes_x_norm_easy = preprocessing.scale(diabetes_x, axis=0)
diabetes_x_norm_easy = preprocessing.normalize(diabetes_x_norm_easy, axis=0)

### Splitting your dataset in training, validation and test sets

Before you star to work with your dataset, split it into a training and test set and never touch the test set again until the very end of the training.

It is also good practice to store all your parameters in a dictionary that you can store and retrieve later.

In [ ]:
config = {
    'trainN': 100, # number of training examples
    'valN': 50, # number of validation examples
    'testN': 50, # number of test examples
}

In [ ]:
# The following line will be written to the config file
print(str(config))

In [ ]:
# actually write the dict to a file
with open('config.txt', 'w') as f:
    f.write(str(config))

We can now restore the config from the disc

In [ ]:
del(config)

In [ ]:
import ast

In [ ]:
# Load config file from disc
with open('config.txt', 'r') as f:
    config = ast.literal_eval(f.readline())
print(config)

In [ ]:
# Number of training + validation examples
trainValN = config['trainN'] + config['valN']

Before we split our dataset we should first shuffle is in case it is sorted

In [ ]:
shuffled_indx = np.random.permutation(range(len(diabetes_x)))

In [ ]:
diabetes_x_shuffled = diabetes_x_norm[shuffled_indx]
diabetes_y_shuffled = diabetes_y[shuffled_indx]

Split your data into training and test set

In [ ]:
diabetes_x_train_ = diabetes_x_norm[:trainValN]
diabetes_y_train_ = diabetes_y[:trainValN]

diabetes_x_test  = diabetes_x_norm[trainValN:trainValN+config['testN']]
diabetes_y_test  = diabetes_y[trainValN:trainValN+config['testN']]

For model selection (different preprocessing, hyperparameter scan) split your training set again in a training and validation set that you can use freely during training   

In [ ]:
diabetes_x_train = diabetes_x_train_[:config['trainN']]
diabetes_y_train = diabetes_y_train_[:config['trainN']]

diabetes_x_val = diabetes_x_train_[config['trainN']:config['trainN']+config['valN']]
diabetes_y_val = diabetes_y_train_[config['trainN']:config['trainN']+config['valN']]

### Regression -  Ridge Regression and model selection

The first task we want to solve is linear regression. In linear regression, we are given *N* training samples of dimensionality *M* and usually a one dimensional corresponding real valued target variable. 

In [ ]:
from sklearn import linear_model

In [ ]:
def eval_lin_model(x, y, estimator):
    return np.mean((estimator.predict(x)-y)**2)

Lets start with least square regression by setting the regularization parameter *alpha* to zero. 

In [ ]:
config['lsm_alpha'] = 0

In [ ]:
estimator = linear_model.Ridge(alpha=config['lsm_alpha'])
estimator.fit(diabetes_x_train, diabetes_y_train)
lsm_coefs = estimator.coef_
lsm_train_error = eval_lin_model(diabetes_x_train, diabetes_y_train, estimator)
lsm_val_error = eval_lin_model(diabetes_x_val, diabetes_y_val, estimator)

Save the model parameters to disc for later use

In [ ]:
np.save('lsm_model_parameters', lsm_coefs)

Now, lets see how different values of *alpha* effect the model

In [ ]:
alphas = np.logspace(-5,5,500)

**Exercise 1.2:** Fill in the blank fields:

In [ ]:
ridge_coefs = []
ridge_train_errors = []
ridge_val_errors = []
for a in alphas:
    estimator = # Create a Ridge estimator object that takes a as input parameter
    # Fit the model to the data
    ridge_coefs.append() # extract the fitted coefficients from the model
    ridge_train_errors.append() # Append the training error of the current model
    ridge_val_errors.append() # Append the validation error of the current model
min_val_err_indx = np.argmin(ridge_val_errors)

In [ ]:
ridge_coefs = []
ridge_train_errors = []
ridge_val_errors = []
for a in alphas:
    estimator = linear_model.Ridge(alpha=a)
    estimator.fit(diabetes_x_train, diabetes_y_train)
    ridge_coefs.append((a, estimator.coef_))
    ridge_train_errors.append((a, eval_lin_model(diabetes_x_train, diabetes_y_train, estimator)))
    ridge_val_errors.append((a, eval_lin_model(diabetes_x_val, diabetes_y_val, estimator)))

ridge_train_errors = np.asarray(ridge_train_errors)
ridge_val_errors = np.asarray(ridge_val_errors)

min_val_err_indx = np.argmin(ridge_val_errors[:,1])

In [ ]:
fig = plt.figure()
plt.plot(ridge_train_errors[:,0],ridge_train_errors[:,1], 'b', label='ridge train')
plt.plot(ridge_val_errors[:,0],ridge_val_errors[:,1], 'r', label='ridge val')
plt.plot([alphas[0], alphas[-1]], [lsm_train_error]*2, 'b--', label='lsm train')
plt.plot([alphas[0], alphas[-1]], [lsm_val_error]*2, 'r--', label='lsm val')
plt.plot(ridge_val_errors[min_val_err_indx, 0], ridge_val_errors[min_val_err_indx, 1], 'k.', markersize=10)
plt.xscale('log')
plt.xlabel('alpha')
plt.ylabel('error')
plt.legend()
_ = plt.title('Ridge train/val errors as a function of the regularization')

In [ ]:
fig = plt.figure()
plt.plot(*zip(*ridge_coefs))
plt.axvline(ridge_val_errors[min_val_err_indx, 0], c='k', ls='--')
plt.xscale('log')
plt.xlim(plt.xlim()[::-1])  # reverse axis
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
_ = plt.axis('tight')

Further reading: lasso[1], cross-validation[2], grid search[3]

[1] http://scikit-learn.org/stable/modules/linear_model.html#lasso

[2] http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

[3] http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

### Classification - Logistic regression

In [ ]:
shuffled_indx = np.random.permutation(len(iris.data))

iris_x_shuffled = iris_x[shuffled_indx]
iris_y_shuffled = iris_y[shuffled_indx]

iris_x_train = iris_x_shuffled[:100, :2]  # we only take the first two features.
iris_y_train = iris_y_shuffled[:100]

iris_x_test = iris_x_shuffled[100:, :2]
iris_y_test = iris_y_shuffled[100:]

In [ ]:
h = .02  # step size in the mesh

In [ ]:
estimator = linear_model.LogisticRegression(C=1e5)

**Exercise 1.3:** Fit the training data (X,Y): 

In [ ]:
estimator.fit(iris_x_train, iris_y_train)

Plot the decision boundary. For that, we will assign a color to each point in the mesh [x_min, x_max]x[y_min, y_max].

In [ ]:
x_min, x_max = iris_x_train[:, 0].min() - .5, iris_x_train[:, 0].max() + .5
y_min, y_max = iris_x_train[:, 1].min() - .5, iris_x_train[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.c_[xx.ravel(), yy.ravel()]

**Task:** Predict the class for each element in the grid:

In [ ]:
Z = 

In [ ]:
Z = estimator.predict(grid)

In [ ]:
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

plt.scatter(iris_x_train[:, 0], iris_x_train[:, 1], c=iris_y_train, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
_ = plt.yticks(())

Now we can predict samples from the train set

In [ ]:
random_indx = np.random.choice(iris_x_test.shape[0], 10, replace=False)
pred = estimator.predict(iris_x_test[random_indx])
conf = np.asarray([x[pred[i]] for i, x in enumerate(estimator.predict_proba(iris_x_test[random_indx]))])
ground_truth = iris_y_test[random_indx]

pd.DataFrame(np.concatenate([pred[:,None], ground_truth[:,None], conf[:,None]], axis=1), columns=['prediction', 'ground truth', 'confidence'])

### Support vector Machines

In [ ]:
from sklearn import svm

Create training data

In [ ]:
np.random.seed(0)
X = np.r_[np.random.randn(20, 2) - [2, 2], np.random.randn(20, 2) + [2, 2]]
Y = [0] * 20 + [1] * 20

In [ ]:
X.shape

In [ ]:
fig = plt.figure()
_ = plt.scatter(X[:,0], X[:,1], c=Y, cmap=plt.cm.Paired)

In [ ]:
estimator = svm.SVC(kernel='linear', C=1)
estimator.fit(X, Y)

Get hyperplane

In [ ]:
w = estimator.coef_[0]
a = -w[0] / w[1]

In [ ]:
xx = np.linspace(-5, 5)
yy = a * xx - (estimator.intercept_[0]) / w[1]

margin = 1 / np.sqrt(np.sum(estimator.coef_ ** 2))
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

In [ ]:
fig = plt.figure(1)
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')

plt.scatter(estimator.support_vectors_[:, 0], estimator.support_vectors_[:, 1], s=80,
            facecolors='none', zorder=10, edgecolors='k')
plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,
            edgecolors='k')
plt.axis('tight')
x_min = -4.8
x_max = 4.2
y_min = -6
y_max = 6

XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
Z = estimator.predict(np.c_[XX.ravel(), YY.ravel()])

# Put the result into a color plot
Z = Z.reshape(XX.shape)
plt.figure(1)
plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired)

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.xticks(())
_ = plt.yticks(())